# Installation (from our last check in)

In [1]:
# !pip uninstall vtk-osmesa -y # only if alreay ran installs 

In [2]:
# !pip install pyvista geovista trame
# !pip uninstall vtk -y && pip install vtk_osmesa --extra-index=https://wheels.vtk.org

In [3]:
from intake import open_catalog
import xarray as xr
import geovista as gv

## smoke test

In [4]:
# import pyvista as pv
# pv.Cone().plot()

In [5]:
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")
grid = cat['LLC4320_grid'].to_dask()
sst = cat['LLC4320_SST'].to_dask()
ds = xr.merge([grid, sst])
ds

<xarray.Dataset>
Dimensions:  (face: 13, j: 4320, i: 4320, k_p1: 2, j_g: 4320, i_g: 4320,
              time: 9030)
Coordinates: (12/37)
    CS       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    PHrefC   float32 ...
    PHrefF   (k_p1) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    SN       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    XC       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    ...       ...
    k_u      int64 ...
    rA       (face, j, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAs      (face, j_g, i) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAw      (face, j, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    rAz      (face, j_g, i_g) float32 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
  * time     (time) datetime64[ns] 2011-09-13 ... 2012-09-23T05:00:00
Data variables:
    SST      (time, face, j, i) float32 dask.array<chunksize=(1, 1, 4320, 4320), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      Created by calling `open_mdsdataset(llc_method='smallchunks...
    source:       MITgcm
    title:        netCDF wrapper of MITgcm MDS binary data

In [ ]:
da_subsampled = ds.SST.isel(i=slice(0,None,100), j=slice(0,None, 100), time=0).load()
da_subsampled

In [ ]:
da = da_subsampled

plotter = gv.GeoPlotter()
time = 0
for i, face in enumerate(ds.face.values):
    print(i)
    da_face = da.isel(face=0)
    mesh = gv.Transform.from_2d(da_face.XC, da_face.YC, data=da_face.data)
    mesh = mesh.threshold()  # Remove NaN (continents)
    plotter.add_mesh(mesh, show_edges=True, cmap='gist_ncar')

plotter.add_base_layer(texture=gv.natural_earth_1())
plotter.add_coastlines(resolution="10m")
plotter.view_xy()
plotter.add_axes()
plotter.show()

## The above crashes. Lets reproduce exactly what Bane did

In [6]:
import xarray as xr
import geovista as gv
ds = xr.open_dataset('bane_llc_sample.nc')
ds

<xarray.Dataset>
Dimensions:  (face: 13, j: 44, i: 44)
Coordinates: (12/20)
    CS       (face, j, i) float32 ...
    Depth    (face, j, i) float32 ...
    PHrefC   float32 ...
    SN       (face, j, i) float32 ...
    XC       (face, j, i) float32 ...
    YC       (face, j, i) float32 ...
    ...       ...
  * j        (j) int64 0 100 200 300 400 500 ... 3800 3900 4000 4100 4200 4300
    k        int64 ...
    k_l      int64 ...
    k_u      int64 ...
    rA       (face, j, i) float32 ...
    time     datetime64[ns] ...
Data variables:
    SST      (face, j, i) float32 ...

In [ ]:
plotter = gv.GeoPlotter()

for i, face in enumerate(ds.face.values):
    da = ds[dict(face=i)].SST
    mesh = gv.Transform.from_2d(da.XC, da.YC, data=da.data)
    mesh = mesh.threshold()  # Remove NaN (continents)
    plotter.add_mesh(mesh, show_edges=True, cmap='gist_ncar')
    break

plotter.add_base_layer(texture=gv.natural_earth_1())
plotter.add_coastlines(resolution="10m")
plotter.view_xy()
plotter.add_axes()
plotter.show()

In [2]:
import pyvista as pv

In [ ]:
import pyvista as pv
pv.Cone().plot()

In [1]:
import os
os.environ['JUPYTER_IMAGE']

'pangeo/pangeo-notebook:2023.05.18'